In [24]:
import pandas as pd
import numpy as np

In [25]:
dfs_by_date = {}


filename = f'./01_raw_data/vechicle/2019년_12월_자동차_등록자료_통계.xlsx'
        
try:
   df = pd.read_excel(filename, sheet_name='10.연료별_등록현황', header=None)
   dfs_by_date[(2019, 12)] = df 
   print(f'{filename} 불러오기 성공')
except FileNotFoundError:
   print(f'{filename} 파일이 존재하지 않음')
except Exception as e:
    print(f'{filename} 오류 발생: {e}')
        

for year in range(2020, 2025):
    for month in range(1, 13):
        year_str = str(year)
        month_str = f'{month:02d}'
        filename = f'./01_raw_data/vechicle/{year_str}년_{month_str}월_자동차_등록자료_통계.xlsx'
        
        try:
            df = pd.read_excel(filename, sheet_name='10.연료별_등록현황', header=None)
            dfs_by_date[(year, month)] = df 
            print(f'{filename} 불러오기 성공')
        except FileNotFoundError:
            print(f'{filename} 파일이 존재하지 않음')
        except Exception as e:
            print(f'{filename} 오류 발생: {e}')

./01_raw_data/vechicle/2019년_12월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_01월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_02월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_03월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_04월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_05월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_06월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_07월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_08월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_09월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_10월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_11월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2020년_12월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2021년_01월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2021년_02월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2021년_03월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/vechicle/2021년_04월_자동차_등록자료_통계.xlsx 불러오기 성공
./01_raw_data/

In [26]:
dfs_by_date[(2019, 12)]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,NaN,NaN,<,연료별,차종별,용도별,등록현황,NaN,>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,조회년월:,2019.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,연료별,시도별,NaN,서울,부산,대구,인천,광주,대전,울산,...,경기,강원,충북,충남,전북,전남,경북,경남,제주,계
3,NaN,종별,용도별,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,NaN,NaN,사업용,4960,8786,1934,5174,1788,1613,2122,...,11007,1641,2685,2898,2135,4698,4934,5371,645,62554
289,NaN,NaN,계,8204,10512,3005,7096,2701,2490,2753,...,18130,3192,4081,4856,3832,7008,7964,7902,1358,95464
290,NaN,소계,비사업용,2916539,1273439,1138123,1289481,643644,644243,544144,...,5536740,757363,802069,1078953,888198,923432,1397777,1635870,370080,22000659
291,NaN,NaN,사업용,207618,121744,52031,345842,32637,29656,21495,...,228952,25337,35533,39164,40863,132807,48485,85633,226135,1676707


In [27]:
df_list = {}

def preprocess(df_ori, list):
    df = df_ori.copy()
    ym = df.iloc[1, 1]
    year, month = ym.split('.')
    df.insert(0, '월', month)
    df.insert(0, '년도', year)
    df = df.drop(index=range(0, 3), axis=0)
    df.columns = ['년도', '월','연료', '차종', '용도', '서울','부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '전국']
    df[['연료','차종']] = df[['연료', '차종']].ffill()
    df.dropna(subset=df.columns[5:], how='all', inplace=True) 
    df = df.reset_index(drop=True)
    list[(int(year), int(month))] = df 
    return df
    

In [28]:
df_full = pd.DataFrame()

for year in range(2019, 2025):
    for month in range(1, 13):
        if (year, month) in dfs_by_date:
            df_temp = dfs_by_date[(year, month)]
            df_temp = preprocess(df_temp, df_list)
            for region in ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
                            '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주','전국']:
                    df_region = df_temp[['년도', '월', region, '연료', '차종', '용도']].copy()
                    df_region = df_region.rename(columns={region: '등록수'})
                    df_region['지역'] = region 
                    
                    df_full = pd.concat([df_full, df_region], ignore_index=True)
                    
df_full = df_full[['년도', '월', '지역', '연료', '차종', '용도', '등록수']]
df_full = df_full.reset_index(drop=True) 
df_full

,년도,월,지역,연료,차종,용도,등록수
0,2019,12,서울,휘발유,승용,비사업용,1585124
1,2019,12,서울,휘발유,승용,사업용,19838
2,2019,12,서울,휘발유,승용,계,1604962
3,2019,12,서울,휘발유,승합,비사업용,980
4,2019,12,서울,휘발유,승합,사업용,22
...,...,...,...,...,...,...,...
281065,2024,12,전국,총계,특수,사업용,68210
281066,2024,12,전국,총계,특수,계,143890
281067,2024,12,전국,총계,소계,비사업용,24337361
281068,2024,12,전국,총계,소계,사업용,1960558


In [29]:
df_full['date'] = pd.to_datetime(df_full['년도'].astype(str) + '-' +df_full['월'].astype(str) + '-01')

df_full = df_full.sort_values(['지역', '연료', '차종', '용도', 'date'])

df_full['순증가량'] = (
    df_full
    .groupby(['지역', '연료', '차종', '용도'])['등록수']
    .diff()
    .fillna(0)
    .astype(int)
)
df_full = df_full.sort_values(['년도', '월'], ascending=[True, True]).reset_index(drop=True)
df_full = df_full[df_full['date'] >= '2020-01-01']
df_full = df_full.drop(columns=['date'])
df_full.to_csv('./02_interim_data/vechicle_full_data.csv', index=False, encoding='utf-8-sig')

C:\Users\junjo\AppData\Local\Temp\ipykernel_105540\1949304915.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)


In [30]:
df_ev = df_full[df_full['연료'] == '전기']
df_total = df_full[df_full['연료'] == '총계']
df_hydrogen = df_full[df_full['연료'] == '수소']
df_ev = df_ev.reset_index(drop=True) 
df_total = df_total.reset_index(drop=True)
df_hydrogen = df_total.reset_index(drop=True) 
df_ev.to_csv('./02_interim_data/vechicle_ev_data.csv', index=False, encoding='utf-8-sig')
df_total.to_csv('./02_interim_data/vechicle_total_data.csv', index=False, encoding='utf-8-sig')
df_hydrogen.to_csv('./02_interim_data/vechicle_hydrogen_data.csv', index=False, encoding='utf-8-sig')


In [ ]:

hyb = df_full[df_full['연료'].str.contains('하이브리드', na=False)].copy()
hyb['연료'] = '하이브리드'

hybrid_agg = (
    hyb
    .groupby(
        ['년도','월','지역','차종','용도','연료'],
        as_index=False
    )
    [['등록수','순증가량']]
    .sum()
)
df_hyb = hybrid_agg [hybrid_agg ['연료']=='하이브리드'].reset_index(drop=True)
df_hyb.to_csv('./02_interim_data/vechicle_hyb_data.csv', index=False, encoding='utf-8-sig')

,년도,월,지역,차종,용도,연료,등록수,순증가량
0,2020,01,강원,소계,계,하이브리드,14145,229
1,2020,01,강원,소계,비사업용,하이브리드,14098,225
2,2020,01,강원,소계,사업용,하이브리드,47,4
3,2020,01,강원,승용,계,하이브리드,14145,229
4,2020,01,강원,승용,비사업용,하이브리드,14098,225
...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,하이브리드,2,1
16196,2024,12,충북,특수,사업용,하이브리드,0,0
16197,2024,12,충북,화물,계,하이브리드,84,5
16198,2024,12,충북,화물,비사업용,하이브리드,83,4
